In [554]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [555]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [556]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [557]:
metrics = loadMetricData('sarahYee', "metrics")
metrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-04 23:55:00,71,0,0.9,0.000067,93.2,93.2
1436,2016-05-04 23:56:00,69,0,1.0,0.000067,94.1,94.1
1437,2016-05-04 23:57:00,68,0,0.9,0.000069,94.1,94.1
1438,2016-05-04 23:58:00,66,0,1.0,0.000071,94.1,94.1
1439,2016-05-04 23:59:00,65,0,0.9,0.000067,94.1,94.1


In [558]:
metrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [559]:
# clean metric data

metrics.timestamp = pd.to_datetime(metrics.timestamp)

metrics.heartrate = metrics.heartrate.fillna(0)
metrics.steps = metrics.steps.fillna(0)
metrics.calories = metrics.calories.fillna(0)
metrics.gsr = metrics.gsr.fillna(0)
metrics.skintemp = metrics.skintemp.fillna(0)
metrics.airtemp = metrics.airtemp.fillna(0)

# metrics.head(5)

In [560]:
metrics = metrics[metrics.heartrate != 0]
metrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
705,2016-04-22 11:45:00,83,478,1167.40002,0.000000,0.0,0.0
706,2016-04-22 11:46:00,80,0,1.60000,0.000054,89.6,88.7
708,2016-04-22 11:48:00,120,0,3.00000,0.000000,0.0,0.0
709,2016-04-22 11:49:00,82,0,1.10000,0.000000,0.0,0.0
710,2016-04-22 11:50:00,81,0,1.30000,0.000064,88.7,87.8


In [561]:
activities = loadMetricData('sarahYee', "activities")
# activities.tail(5)

In [562]:
sleep = loadMetricData('sarahYee', "sleep")
# sleep.tail(5)

In [563]:
# sleep.dtypes

In [564]:
journal = loadMetricData('sarahYee', "journal")
# journal.tail(5)

In [565]:
# journal.dtypes

In [566]:
# clean metric data
journal.starttime= pd.to_datetime(journal.starttime)
journal.endtime= pd.to_datetime(journal.endtime)

journal.qualitative = journal.qualitative.fillna(0)
journal.excited = journal.excited.fillna(0)
journal.happy = journal.happy.fillna(0)
journal.calm = journal.calm.fillna(0)
journal.anxious = journal.anxious.fillna(0)
journal.sad = journal.sad.fillna(0)
journal.angry = journal.angry.fillna(0)
journal.hungry = journal.hungry.fillna(0)
journal.tired = journal.tired.fillna(0)
journal.bored = journal.bored.fillna(0)
journal.ansaan = journal.ansaan.fillna(0)
journal.exhaca = journal.exhaca.fillna(0)

print(journal.shape)
journal.head(5)
# journal.tail(5)

(127, 24)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,0,0,0,6,0,0,6,6
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,0,0,0,6,0,0,6,6
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,0,0,0,0,10,0,10,10,0,0
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,0,0,0,0,5,3,5,0,3,3
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,0,0,0,0,0,0,0,0,0,0


In [567]:

journal['boolexcited'] =  journal['excited'] > 0
journal['boolhappy'] =  journal['happy'] > 0
journal['boolcalm'] =  journal['calm'] > 0
journal['boolanxious'] =  journal['anxious'] > 0
journal['boolsad'] =  journal['sad'] > 0
journal['boolangry'] =  journal['angry'] > 0
journal['booltired'] =  journal['tired'] > 0
journal['boolhungry'] =  journal['hungry'] > 0
journal['boolbored'] =  journal['bored'] > 0
journal['exhaca'] =  journal['exhaca'] > 0
journal['ansaan'] =  journal['ansaan'] > 0

journal['exha'] =  journal['exha'] > 0
journal['haca'] =  journal['haca'] > 0
journal['anan'] =  journal['anan'] > 0
journal['ansa'] =  journal['anan'] > 0

    
boolexcited = journal.iloc[:, 18].values.reshape(journal.shape[0], 1)
boolhappy = journal.iloc[:, 19].values.reshape(journal.shape[0], 1)
boolcalm = journal.iloc[:, 20].values.reshape(journal.shape[0], 1)
boolanxious = journal.iloc[:, 21].values.reshape(journal.shape[0], 1)
boolsad = journal.iloc[:, 22].values.reshape(journal.shape[0], 1)
boolangry = journal.iloc[:, 23].values.reshape(journal.shape[0], 1)
booltired = journal.iloc[:, 24].values.reshape(journal.shape[0], 1)
boolhungry = journal.iloc[:, 25].values.reshape(journal.shape[0], 1)
boolbored = journal.iloc[:, 26].values.reshape(journal.shape[0], 1)
exhaca = journal.iloc[:, 27].values.reshape(journal.shape[0], 1)
ansaan = journal.iloc[:, 28].values.reshape(journal.shape[0], 1)
exha = journal.iloc[:, 29].values.reshape(journal.shape[0], 1)
haca = journal.iloc[:, 30].values.reshape(journal.shape[0], 1)
anan = journal.iloc[:, 31].values.reshape(journal.shape[0], 1)
ansa = journal.iloc[:, 32].values.reshape(journal.shape[0], 1)

journal.head(5)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,False,False,True,False,False,False,False,False,False,False
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,True,True,False,False,True,False,False,False,False,False
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,False,False,False,False,False,False,False,False,False,False


In [568]:
# for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['ex'] = (0< sarahJournal.iloc[i, 18].astype(int))
#     sarahJournal['ha'] = sarahJournal.iloc[i, 19]== True
#     sarahJournal['ca'] = sarahJournal.iloc[i, 20]== True
#     sarahJournal['ys'] = sarahJournal['ex'].astype(int) + sarahJournal['ha'].astype(int) + sarahJournal['ca'].astype(int)

# print(sarahJournal['ex'])
    

In [569]:
# def makeEmoIndicators(newvar, journalData, journal_emo1_loc, journal_emo2_loc, journal_emo3_loc):
  
#     journalData[newvar] = False
    
#     for i in range(0, journalData.shape[0]):
#         journalData['emo1'] = (journalData.iloc[i, journal_emo1_loc]) == True)
#         journalData['emo2'] = (journalData.iloc[i, journal_emo2_loc] == True)
#         journalData['emo3'] = (journalData.iloc[i, journal_emo3_loc] == True)
#         journalData['ysum'] = journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int)
# #         print(journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int))
#         journalData.loc[journalData['ysum']!=0, newvar] = True
        
#     print(journalData['emo1'].values)
#     journalData = journalData.drop(['emo1', 'emo2', 'emo3', 'ysum'], axis=1)
#     return journalData

In [570]:
# sarahJournal.dtypes

In [571]:
# testData= makeEmoIndicators('exhaca', sarahJournal, 18, 19, 20)
# # print(testData.describe())

In [572]:
# sarahJournal.boolexhapcal.convert_objects(convert_numeric=True)

In [573]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [574]:
# indicator for walking
totalData = makeIndicators('excitement', metrics, journal, 0, 1, 18)
totalData = makeIndicators('happy', metrics, journal, 0, 1, 19)
totalData = makeIndicators('calm', metrics, journal, 0, 1, 20)
totalData = makeIndicators('anxious', metrics, journal, 0, 1, 21)
totalData = makeIndicators('sad', metrics, journal, 0, 1, 22)
totalData = makeIndicators('angry', metrics, journal, 0, 1, 23)
totalData = makeIndicators('hungry', metrics, journal, 0, 1, 24)
totalData = makeIndicators('tired', metrics, journal, 0, 1, 25)
totalData = makeIndicators('bored', metrics, journal, 0, 1, 26)

totalData = makeIndicators('exhaca', metrics, journal, 0, 1, 27)
totalData = makeIndicators('ansaan', metrics, journal, 0, 1, 28)

totalData = makeIndicators('exha', metrics, journal, 0, 1, 29)
totalData = makeIndicators('haca', metrics, journal, 0, 1, 30)
totalData = makeIndicators('anan', metrics, journal, 0, 1, 31)
totalData = makeIndicators('ansa', metrics, journal, 0, 1, 32)




In [575]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  17279.000000  17279.000000  17279.000000  17279.000000  17279.000000   
mean      87.660281      8.785231      1.822420      0.045547     88.762446   
std       18.073892     26.172463      8.982735      0.417669      6.415743   
min       57.000000      0.000000      0.000000      0.000000      0.000000   
25%       71.000000      0.000000      1.000000      0.000054     86.900000   
50%       87.000000      0.000000      1.200000      0.000062     89.600000   
75%       99.000000      0.000000      1.800000      0.000081     91.400000   
max      168.000000    695.000000   1167.400020     10.500000     97.700000   

            airtemp excitement     happy       calm   anxious    ...      \
count  17279.000000      17279     17279      17279     17279    ...       
mean      85.298235   0.184038  0.245327  0.0478037   0.17773    ...       
std        7.025071   0.387527  0.430293   0.213357  0.38229

In [576]:
print(totalData.head())

              timestamp  heartrate  steps    calories       gsr  skintemp  \
705 2016-04-22 11:45:00         83    478  1167.40002  0.000000       0.0   
706 2016-04-22 11:46:00         80      0     1.60000  0.000054      89.6   
708 2016-04-22 11:48:00        120      0     3.00000  0.000000       0.0   
709 2016-04-22 11:49:00         82      0     1.10000  0.000000       0.0   
710 2016-04-22 11:50:00         81      0     1.30000  0.000064      88.7   

     airtemp excitement  happy   calm  ...    angry hungry  tired  bored  \
705      0.0      False  False  False  ...    False  False  False  False   
706     88.7      False  False  False  ...    False  False  False  False   
708      0.0      False  False  False  ...    False  False  False  False   
709      0.0      False  False  False  ...    False  False  False  False   
710     87.8      False  False  False  ...    False  False  False  False   

    exhaca ansaan   exha   haca   anan   ansa  
705  False  False  False  False 

In [577]:
totalData.to_csv('sarahTotal.csv')

In [578]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
705,2016-04-22 11:45:00,83,478,1167.40002,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
706,2016-04-22 11:46:00,80,0,1.60000,0.000054,89.6,88.7,False,False,False,...,False,False,False,False,False,False,False,False,False,False
708,2016-04-22 11:48:00,120,0,3.00000,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
709,2016-04-22 11:49:00,82,0,1.10000,0.000000,0.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
710,2016-04-22 11:50:00,81,0,1.30000,0.000064,88.7,87.8,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [579]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(17279, 26)
              timestamp  heartrate  steps    calories       gsr  skintemp  \
705 2016-04-22 11:45:00         83    478  1167.40002  0.000000       0.0   
706 2016-04-22 11:46:00         80      0     1.60000  0.000054      89.6   
708 2016-04-22 11:48:00        120      0     3.00000  0.000000       0.0   
709 2016-04-22 11:49:00         82      0     1.10000  0.000000       0.0   
710 2016-04-22 11:50:00         81      0     1.30000  0.000064      88.7   

     airtemp excitement  happy   calm ... exhaca ansaan   exha   haca   anan  \
705      0.0      False  False  False ...  False  False  False  False  False   
706     88.7      False  False  False ...  False  False  False  False  False   
708      0.0      False  False  False ...  False  False  False  False  False   
709      0.0      False  False  False ...  False  False  False  False  False   
710     87.8      False  False  False ...  False  False  False  False  False   

      ansa weekday month hour min  
705  Fal

In [580]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
# sally = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
# barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
# sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
# arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
# work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
# home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 7].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 8].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 9].values.reshape(totalData.shape[0], 1)

In [581]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         thesis, 
#         sally, 
#         barb, 
#         sol, 
#         arash, 
#         work, 
#         home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(17279, 9)

# Positive ( Excited, Happy, Calm)

In [582]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(17279,)

In [583]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [584]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [585]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

## Positive ( Excited, Happy, Calm) Model Results

In [586]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0115750310045 0.0123456790123
 Accuracy Rate  0.988424968995  0.987654320988
True Positives  0.981454005935  0.980613893376
True Negatives              --   0.98986315256


(0.012345679012345678,
 0.98765432098765427,
 0.98061389337641358,
 0.98986315255955393)

In [627]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=5000, alpha=0.0001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0420835055808 0.0488040123457
      Accuracy Rate  0.957916494419  0.951195987654
     True Positives   0.56005788712  0.493103448276
     True Negatives              --  0.978340825501


(0.048804012345679014,
 0.95119598765432101,
 0.49310344827586206,
 0.97834082550061297)

In [630]:
print(clf.coef_)
print(clf.intercept_)
print(X_train_std[0:5])
print(X_train[0:5])
# heartrate, steps, calories, gsr, skintemp, airtemp, weekday, hour, curmin


[[-0.22892829 -0.30019283 -0.17956911 -1.46410098  1.71595724 -2.17994176
   1.2145938   1.52692676 -1.04897758]]
[-1.97390115]
[[  3.52759580e+00   0.00000000e+00   1.02846809e-01   1.47285951e-04
    1.46583562e+01   1.32235531e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00]
 [  4.07878264e+00   0.00000000e+00   9.34970994e-02   1.54699673e-04
    1.47999068e+01   1.33524935e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00]
 [  5.84258054e+00   4.60966496e-01   1.30895939e-01   1.57170914e-04
    1.42337042e+01   1.25788512e+01   0.00000000e+00   2.33674548e+00
    0.00000000e+00]
 [  4.51973211e+00   0.00000000e+00   1.02846809e-01   1.24056288e-04
    1.21104445e+01   1.05158050e+01   2.58956568e+00   2.33674548e+00
    4.76290967e+00]
 [  4.68508817e+00   0.00000000e+00   1.49595359e-01   1.41602097e-04
    1.33844003e+01   1.16762684e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00]]
[[  6.40000000e+01   0.00000000e+00   1.10000000e+00   5.96000000e-05
 

In [588]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0367920628359 0.0341435185185
 Accuracy Rate  0.963207937164  0.965856481481
True Positives             1.0             1.0
True Negatives              --  0.955144450076


(0.034143518518518517, 0.96585648148148151, 1.0, 0.95514445007602633)

In [589]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0128978916908 0.0115740740741
 Accuracy Rate  0.987102108309  0.988425925926
True Positives  0.996661721068  0.999192245557
True Negatives              --  0.985048150025


(0.011574074074074073,
 0.98842592592592593,
 0.99919224555735053,
 0.98504815002534207)

# Negative ( Anxious, Sad, Angry)

In [590]:
# define y
y = totalData.ansaan.values
y.shape

(17279,)

In [591]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [592]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [593]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0423315419595 0.0435956790123
 Accuracy Rate  0.957668458041  0.956404320988
True Positives 0.0589353612167 0.0313901345291
True Negatives              --  0.997984277363


(0.043595679012345678,
 0.95640432098765427,
 0.031390134529147982,
 0.99798427736343476)

In [594]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set      Test Set   
------------------- -------------- ---------------
         Error Rate 0.032244729227 0.0316358024691
      Accuracy Rate 0.967755270773  0.968364197531
     True Positives 0.349809885932   0.37668161435
     True Negatives             --  0.994960693409


(0.031635802469135804,
 0.96836419753086422,
 0.37668161434977576,
 0.99496069340858695)

In [595]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0455560148822 0.0459104938272
 Accuracy Rate  0.954443985118  0.954089506173
True Positives  0.121673003802  0.116591928251
True Negatives              --   0.99173553719


(0.045910493827160496,
 0.9540895061728395,
 0.11659192825112108,
 0.99173553719008267)

In [596]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0440677966102 0.0435956790123
 Accuracy Rate   0.95593220339  0.956404320988
True Positives             0.0             0.0
True Negatives              --  0.999395283209


(0.043595679012345678, 0.95640432098765427, 0.0, 0.99939528320903048)

# Happy/Calm 

In [597]:
# define y
y = totalData.haca.values
y.shape

(17279,)

In [598]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [599]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [600]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.075651095494 0.0706018518519
 Accuracy Rate 0.924348904506  0.929398148148
True Positives 0.220243673852  0.230414746544
True Negatives             --  0.993263157895


(0.070601851851851846,
 0.92939814814814814,
 0.2304147465437788,
 0.99326315789473685)

In [601]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0904505994212 0.0914351851852
      Accuracy Rate  0.909549400579  0.908564814815
     True Positives  0.687910028116  0.672811059908
     True Negatives              --  0.930105263158


(0.091435185185185189,
 0.90856481481481477,
 0.67281105990783407,
 0.93010526315789477)

In [602]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0950806118231 0.0960648148148
 Accuracy Rate  0.904919388177  0.903935185185
True Positives  0.419868791003  0.428571428571
True Negatives              --  0.947368421053


(0.096064814814814811,
 0.90393518518518523,
 0.42857142857142855,
 0.94736842105263153)

In [603]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0960727573377 0.0991512345679
 Accuracy Rate  0.903927242662  0.900848765432
True Positives  0.661668228679  0.647465437788
True Negatives              --           0.924


(0.099151234567901231,
 0.9008487654320988,
 0.64746543778801846,
 0.92400000000000004)

# Anxious/Angry

In [604]:
# define y
y = totalData.anan.values
y.shape

(17279,)

In [605]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [606]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Angry Model Results

In [607]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0241422075238 0.0270061728395
 Accuracy Rate  0.975857792476   0.97299382716
True Positives  0.191304347826  0.135593220339
True Negatives              --  0.982634146341


(0.027006172839506171,
 0.97299382716049387,
 0.13559322033898305,
 0.9826341463414634)

In [608]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0095907399752 0.0113811728395
      Accuracy Rate  0.990409260025   0.98861882716
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.011381172839506173, 0.98861882716049387, 0.0, 1.0)

In [609]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0455560148822 0.0515046296296
 Accuracy Rate  0.954443985118   0.94849537037
True Positives  0.417391304348  0.406779661017
True Negatives              --  0.954731707317


(0.051504629629629629,
 0.94849537037037035,
 0.40677966101694918,
 0.95473170731707313)

In [610]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00950806118231 0.0113811728395
 Accuracy Rate   0.990491938818   0.98861882716
True Positives              0.0             0.0
True Negatives               --             1.0


(0.011381172839506173, 0.98861882716049387, 0.0, 1.0)

# Excited/Happy

In [611]:
# define y
y = totalData.exha.values
y.shape

(17279,)

In [612]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [613]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [614]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0769739561802  0.0779320987654
 Accuracy Rate   0.92302604382   0.922067901235
True Positives 0.0042872454448 0.00492610837438
True Negatives              --              1.0


(0.077932098765432098, 0.92206790123456794, 0.0049261083743842365, 1.0)

In [615]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0895411326995 0.0954861111111
      Accuracy Rate  0.910458867301  0.904513888889
     True Positives  0.872454448017  0.891625615764
     True Negatives              --   0.90560904144


(0.095486111111111105,
 0.90451388888888884,
 0.89162561576354682,
 0.90560904143993304)

In [616]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0807771806532 0.0860339506173
 Accuracy Rate  0.919222819347  0.913966049383
True Positives  0.167202572347  0.162561576355
True Negatives              --   0.97781498535


(0.086033950617283944,
 0.91396604938271608,
 0.1625615763546798,
 0.97781498534951861)

In [617]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0768912773874  0.0779320987654
 Accuracy Rate  0.923108722613   0.922067901235
True Positives  0.005359056806 0.00492610837438
True Negatives              --              1.0


(0.077932098765432098, 0.92206790123456794, 0.0049261083743842365, 1.0)

# Anxious/Sad

In [618]:
# define y
y = totalData.ansa.values
y.shape

(17279,)

In [619]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [620]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [621]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate  0.0573790822654   0.056712962963
 Accuracy Rate   0.942620917735   0.943287037037
True Positives 0.00578871201158 0.00344827586207
True Negatives               --   0.998978340826


(0.056712962962962965,
 0.94328703703703698,
 0.0034482758620689655,
 0.99897834082550063)

In [622]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set      Test Set   
------------------- -------------- ---------------
         Error Rate 0.038197602315 0.0451388888889
      Accuracy Rate 0.961802397685  0.954861111111
     True Positives  0.62518089725  0.565517241379
     True Negatives             --  0.977932161831


(0.045138888888888888,
 0.95486111111111116,
 0.56551724137931036,
 0.97793216183081322)

In [623]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0424968995453 0.0491898148148
 Accuracy Rate  0.957503100455  0.950810185185
True Positives  0.552821997106  0.489655172414
True Negatives              --  0.978136493666


(0.049189814814814818,
 0.95081018518518523,
 0.48965517241379308,
 0.97813649366571309)

In [624]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0442331541959 0.0491898148148
 Accuracy Rate  0.955766845804  0.950810185185
True Positives  0.492040520984  0.431034482759
True Negatives              --  0.981610134859


(0.049189814814814818,
 0.95081018518518523,
 0.43103448275862066,
 0.98161013485901105)